In [1]:
import numpy as np
import pandas as pd
import os
import sys

In [2]:
folder_path = '/Downloads/str1627/for_test_v4/'

In [4]:
isescan_cols = [0,2,3,4]
blast_cols = [0,2,3,4,7]

In [5]:
depth_file = pd.read_csv('/home/ignatsonets/Downloads/str1627/for_test_v4/yp_str_1627.aln2.depth', sep = '\t', header = None, names = ['Contig', 'Coord', 'Depth'])
depth_by_contig_df = depth_file.groupby('Contig')['Depth'].median().reset_index()
length_by_contig_df = depth_file.groupby('Contig')['Coord'].size().reset_index()
length_by_contig_df.columns = ['Contig', 'Length']

In [ ]:
#  thresholds also saved here, in case I will get rid of argparse
#threshold_cov = 1.5
#threshold_len = 0.5
for file_name in os.listdir(folder_path):
    if file_name.endswith('.depth'):
        depth_file = pd.read_csv(file_name, sep = '\t', header = None, names = ['Contig', 'Coord', 'Depth'])
        depth_by_contig_df = depth_file.groupby('Contig')['Depth'].median().reset_index()
        length_by_contig_df = depth_file.groupby('Contig')['Coord'].size().reset_index()
        length_by_contig_df.columns = ['Contig', 'Length']

In [ ]:
depth_file

In [ ]:
depth_by_contig_df

In [ ]:
length_by_contig_df

In [24]:
data_isescan = pd.read_csv('/home/ignatsonets/Downloads/str1627/for_test_v4/yp_str_1627.isescan_res.csv', sep =',', usecols = isescan_cols)
data_isescan['Length'] = data_isescan['isEnd'] - data_isescan['isBegin']

In [ ]:
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        data_isescan = pd.read_csv(file_name, sep =',', usecols = isescan_cols)
            #  ISEscan file doesn't have length column, like a BLAST, so we create it.
        data_isescan['Length'] = data_isescan['isEnd'] - data_isescan['isBegin']

In [25]:
data_isescan

,seqID,cluster,isBegin,isEnd,Length
0,contig_2,IS21_203,39,1948,1909
1,contig_4,IS256_47,1,1315,1314
2,contig_4,IS200/IS605_384,4275,4985,710
3,contig_4,IS200/IS605_384,90582,91292,710
4,contig_4,IS200/IS605_384,100781,101494,713
5,contig_5,IS3_61,42715,46332,3617


In [18]:
output_res_isescan =[]
for _, row in data_isescan.iterrows():
    start = row['isBegin']
    stop = row['isEnd']
    window_data = depth_file[(depth_file['Coord'] >= start) & (depth_file['Coord'] <= stop)]
    median_window = window_data['Depth'].median()
    output_res_isescan.append(median_window)
output_res_isescan
data_isescan['Median_Depth']= output_res_isescan

In [19]:
data_isescan

,seqID,cluster,isBegin,isEnd,Median_Depth
0,contig_2,IS21_203,39,1948,90.5
1,contig_4,IS256_47,1,1315,92.0
2,contig_4,IS200/IS605_384,4275,4985,222.0
3,contig_4,IS200/IS605_384,90582,91292,196.0
4,contig_4,IS200/IS605_384,100781,101494,221.0
5,contig_5,IS3_61,42715,46332,209.0


In [22]:
isescan_cov_check_result_df = pd.DataFrame(columns=data_isescan.columns)
for _, row in data_isescan.iterrows():
    name = row['seqID']
    median_of_is = row['Median_Depth']
    median_by_contig = depth_by_contig_df.loc[depth_by_contig_df['Contig'] == name, 'Depth'].values[0]
    if median_of_is >= 1.5 * median_by_contig:
        isescan_cov_check_result_df = isescan_cov_check_result_df.append(row)
isescan_cov_check_result_df
#result_df.to_csv('ISEscan_res_verified.tsv', sep='\t', header=False, index=False)
#print("IS coordinates from ISEscan verified and saved.")

/tmp/ipykernel_2396118/1021279972.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  isescan_cov_check_result_df = isescan_cov_check_result_df.append(row)


,seqID,cluster,isBegin,isEnd,Median_Depth
0,contig_2,IS21_203,39,1948,90.5


In [27]:
isescan_len_check_result_df = pd.DataFrame(columns=data_isescan.columns)
for _, row in data_isescan.iterrows():
    name = row['seqID']
    length_of_determinant = row['Length']
        #  same logic as for coverage, thanks BLAST sukablyat'
    try:
        contig_length = length_by_contig_df.loc[length_by_contig_df['Contig'] == name, 'Length'].values[0]
        if length_of_determinant >= 0.5 * contig_length:
            isescan_len_check_result_df = isescan_len_check_result_df.append(row)
    except:
        pass
isescan_len_check_result_df

/tmp/ipykernel_2396118/3289407179.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  isescan_len_check_result_df = isescan_len_check_result_df.append(row)


,seqID,cluster,isBegin,isEnd,Length
0,contig_2,IS21_203,39,1948,1909


In [14]:
blast_is = pd.read_csv('/home/ignatsonets/Downloads/str1627/for_test_v4/yp_str_1627.blast_ISs.txt', sep = '\t', header = None, usecols = blast_cols, names = ['seqID', 'Length', 'isBegin', 'isEnd', 'Title'])
blast_phage = pd.read_csv('/home/ignatsonets/Downloads/str1627/for_test_v4/yp_str_1627.blast_phage.txt', sep = '\t', header = None, usecols = blast_cols, names = ['seqID', 'Length', 'isBegin', 'isEnd', 'Title'])
blast_rrna = pd.read_csv('/home/ignatsonets/Downloads/str1627/for_test_v4/yp_str_1627.blast_rrnas.txt', sep = '\t', header = None, usecols = blast_cols, names = ['seqID', 'Length', 'isBegin', 'isEnd', 'Title'])

In [15]:
blast_is

,seqID,Length,isBegin,isEnd,Title
0,contig_2,1050,818,1867,lcl|NC_003143.1_cds_WP_002430106.1_1954 [gene=...
1,contig_4,1209,36,1244,lcl|NC_003134.1_cds_WP_002209743.1_4283 [locus...
2,contig_5,1356,45063,46417,lcl|NC_003143.1_cds_WP_100067904.1_4055 [locus...


In [16]:
blast_phage

,seqID,Length,isBegin,isEnd,Title


In [17]:
blast_rrna

,seqID,Length,isBegin,isEnd,Title


In [20]:
output_res_blast =[]
for _, row in blast_is.iterrows():
    start = row['isBegin']
    stop = row['isEnd']
    window_data = depth_file[(depth_file['Coord'] >= start) & (depth_file['Coord'] <= stop)]
    median_window = window_data['Depth'].median()
    output_res_blast.append(median_window)
output_res_blast
blast_is['Median_Depth']= output_res_blast

In [21]:
blast_is

,seqID,Length,isBegin,isEnd,Title,Median_Depth
0,contig_2,1050,818,1867,lcl|NC_003143.1_cds_WP_002430106.1_1954 [gene=...,105.0
1,contig_4,1209,36,1244,lcl|NC_003134.1_cds_WP_002209743.1_4283 [locus...,92.0
2,contig_5,1356,45063,46417,lcl|NC_003143.1_cds_WP_100067904.1_4055 [locus...,221.0


In [23]:
blast_is_cov_check_result_df = pd.DataFrame(columns=blast_is.columns)
for _, row in blast_is.iterrows():
    name = row['seqID']
    median_of_is = row['Median_Depth']
    median_by_contig = depth_by_contig_df.loc[depth_by_contig_df['Contig'] == name, 'Depth'].values[0]
    if median_of_is >= 1.5 * median_by_contig:
        blast_is_cov_check_result_df = blast_is_cov_check_result_df.append(row)
blast_is_cov_check_result_df

/tmp/ipykernel_2396118/3843439744.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  blast_is_cov_check_result_df = blast_is_cov_check_result_df.append(row)


,seqID,Length,isBegin,isEnd,Title,Median_Depth
0,contig_2,1050,818,1867,lcl|NC_003143.1_cds_WP_002430106.1_1954 [gene=...,105.0


In [28]:
blast_is_len_check_result_df = pd.DataFrame(columns=blast_is.columns)
for _, row in blast_is.iterrows():
    name = row['seqID']
    length_of_determinant = row['Length']
        #  same logic as for coverage, thanks BLAST sukablyat'
    try:
        contig_length = length_by_contig_df.loc[length_by_contig_df['Contig'] == name, 'Length'].values[0]
        if length_of_determinant >= 0.5 * contig_length:
            blast_is_len_check_result_df = blast_is_len_check_result_df.append(row)
    except:
        pass
blast_is_len_check_result_df

/tmp/ipykernel_2396118/32199292.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  blast_is_len_check_result_df = blast_is_len_check_result_df.append(row)


,seqID,Length,isBegin,isEnd,Title,Median_Depth
0,contig_2,1050,818,1867,lcl|NC_003143.1_cds_WP_002430106.1_1954 [gene=...,105.0
